# Detection & Classification using machine learning

## Building the dataset 

In [ ]:
seed = 42
image_folder = 'dataset/train/images/'
label_folder = 'dataset/train/labels/'
target_image_folder = 'dataset/train/images_aug/'
target_label_folder = 'dataset/train/labels_aug/'

#### DATA AUGMENTATION

In [ ]:
from src.augment import ImageBatchProcessor
processor = ImageBatchProcessor(image_folder=image_folder, label_folder=label_folder, target_image_folder=target_image_folder, target_label_folder=target_label_folder)
processor.augment_and_save(nb_augmentation=5)

#### Image cropping and negative examples generation

In [ ]:
from src.crop import crop_images_from_folder
crop_images_from_folder(target_image_folder, target_label_folder, 'dataset/train/aug_cropped_images/')
crop_images_from_folder(image_folder, label_folder, 'dataset/train/cropped_images/')
val_path = 'dataset/val/images/'
val_label_path = 'dataset/val/labels/'
crop_images_from_folder(val_path, val_label_path, 'dataset/val/cropped_images/')

#### Dataset building and preprocessing

In [ ]:
from src.dataset import dataset
val_path = 'dataset/val/cropped_images/'
aug_train_path = 'dataset/train/aug_cropped_images/'
train_path = 'dataset/train/cropped_images/'
standard_size = (64, 64)
label_size_factor = 1
train_data = dataset(img_dir=train_path, augment_path=aug_train_path, label_size_factor=label_size_factor, standard_size=standard_size)
val_data = dataset(val_path, train=False, standard_size=standard_size)
label_count = {}
for x in train_data.images:
    label = x.label
    if label in label_count.keys():
        label_count[label] +=1
    else:
        label_count[label] =1
        
for x in val_data.images:
    label = x.label
    if label in label_count.keys():
        label_count[label] +=1
    else:
        label_count[label] =1
label_count

## First Model Training

In [ ]:
from src.model import model
max_iter = 10000
# initialize the model
my_model = model(seed)
#training
print(f"Training model {my_model.name} with seed {seed},max_iter {max_iter}, standard_size {standard_size}, label_size_factor {label_size_factor}")
my_model.train_svm(train_data,val_data, verbose=0, max_iter=max_iter)
accuracy = my_model.evaluate(val_data)

## First Detection on test

In [ ]:
from detection import detection_images_in_folder
import warnings

warnings.filterwarnings("ignore",
                        message="Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present.")

image_path = "dataset/test/images"
detection_images_in_folder(image_path, my_model, 'detections.csv')

## Augmentating train none label with the false positive of other labels

In [ ]:
import pandas as pd
from detection import build_detection_validation
label_path = "dataset/train/labels"
build_detection_validation(label_path)
# Load the data
validations = pd.read_csv('validations.csv', header=None)
detections = pd.read_csv('detections.csv', header=None)
# Filter out the 'ff' labels
validations = validations[validations.iloc[:, 5] != 'ff']
detections = detections[detections.iloc[:, 6] != 'ff']
# Function to check if a value can be converted to float
def is_float(value):
    try:
        float(value)
        return True
    except ValueError:
        print(f"{value} rejected")
        return False
validations = validations[validations.iloc[:, 1:5].map(is_float).all(axis=1)]
detections = detections[detections.iloc[:, 1:5].map(is_float).all(axis=1)]
validations.iloc[:, 1:5] = validations.iloc[:, 1:5].astype(float)
detections.iloc[:, 1:5] = detections.iloc[:, 1:5].astype(float)
detections.iloc[:, 5] = detections.iloc[:, 5].astype(float)

from gen_neg import get_negative_prediction
negative_samples_dir = "dataset/train/cropped_images"
image_folder_path = "dataset/train/images"
get_negative_prediction(detections, validations, image_folder_path, negative_samples_dir)

## Second model training

In [ ]:
#reload and process the datasets
train_data = dataset(img_dir=train_path, augment_path=aug_train_path, label_size_factor=label_size_factor, standard_size=standard_size)
val_data = dataset(val_path, train=False, standard_size=standard_size)
my_model = model(seed)
#training
print(f"Training second model {my_model.name} with seed {seed},max_iter {max_iter}, standard_size {standard_size}, label_size_factor {label_size_factor}")
my_model.train_svm(train_data,val_data, verbose=0, max_iter=max_iter)
accuracy = my_model.evaluate(val_data)

## Second Detection & Augmentation on test


In [ ]:
detection_images_in_folder(image_path, my_model, 'detections.csv')
build_detection_validation(label_path)
detections = pd.read_csv('detections.csv', header=None)
# Filter out the 'ff' labels
detections = detections[detections.iloc[:, 6] != 'ff']
detections = detections[detections.iloc[:, 1:5].map(is_float).all(axis=1)]
detections.iloc[:, 1:5] = detections.iloc[:, 1:5].astype(float)
detections.iloc[:, 5] = detections.iloc[:, 5].astype(float)
get_negative_prediction(detections, validations, image_folder_path, negative_samples_dir)

## Third Model Training

In [ ]:
#reload and process the datasets
train_data = dataset(img_dir=train_path, augment_path=aug_train_path, label_size_factor=label_size_factor, standard_size=standard_size)
val_data = dataset(val_path, train=False, standard_size=standard_size)
my_model = model(seed)
#training
print(f"Training third model {my_model.name} with seed {seed},max_iter {max_iter}, standard_size {standard_size}, label_size_factor {label_size_factor}")
my_model.train_svm(train_data,val_data, verbose=0, max_iter=max_iter)
accuracy = my_model.evaluate(val_data)

## Final detection on validation dataset

In [ ]:
image_path = "dataset/val/images"
label_path = "dataset/val/labels"

detection_images_in_folder(image_path, my_model, 'detections.csv')
build_detection_validation(label_path)
validations = pd.read_csv('validations.csv', header=None)
detections = pd.read_csv('detections.csv', header=None)
validations = validations[validations.iloc[:, 5] != 'ff']
detections = detections[detections.iloc[:, 6] != 'ff']
validations = validations[validations.iloc[:, 1:5].map(is_float).all(axis=1)]
detections = detections[detections.iloc[:, 1:5].map(is_float).all(axis=1)]
validations.iloc[:, 1:5] = validations.iloc[:, 1:5].astype(float)
detections.iloc[:, 1:5] = detections.iloc[:, 1:5].astype(float)
detections.iloc[:, 5] = detections.iloc[:, 5].astype(float)

## Results

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
# Initialize lists to store y (ground truth) and y_pred (predictions)
y = []
y_pred = []
matching_validation_row = []
fp_data = []

# Define IoU function
def iou(box1, box2):
    # Unpack the coordinates
    x1_min, y1_min, x1_max, y1_max = box1
    x2_min, y2_min, x2_max, y2_max = box2

    # Calculate the intersection coordinates
    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)

    # Calculate the intersection area
    inter_area = max(0, inter_x_max - inter_x_min) * max(0, inter_y_max - inter_y_min)

    # Calculate the areas of each box
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)

    # Calculate the union area
    union_area = box1_area + box2_area - inter_area

    # Calculate the IoU
    iou = inter_area / union_area

    return iou

# Match detections with ground truth
for _, detection_row in detections.iterrows():
    image_id_d, x_min_d, y_min_d, x_max_d, y_max_d, score_d, label_d = detection_row
    max_iou = 0.5
    matching_validation = 'none'

    for idx, validation_row in validations.iterrows():
        image_id_v, x_min_v, y_min_v, x_max_v, y_max_v, label_v = validation_row
        
        if image_id_v == image_id_d and label_v == label_d:
            iou_value = iou((x_min_v, y_min_v, x_max_v, y_max_v), (x_min_d, y_min_d, x_max_d, y_max_d))
            
            if iou_value > max_iou:
                max_iou = iou_value
                matching_validation = label_v
                
                matching_validation_row.append(idx)
    
    # Add ground truth and prediction based on maximum IoU
    y_pred.append(label_d)
    y.append(matching_validation)
    
    if matching_validation == 'none':
        fp_data.append(list(detection_row))

for idx, validation_row in validations.iterrows():
    image_id_v, x_min_v, y_min_v, x_max_v, y_max_v, label_v = validation_row
    if idx not in matching_validation_row:
        y.append(label_v)
        y_pred.append('none')

# Plot confusion matrix
cm = confusion_matrix(y, y_pred, normalize='true')
# Get unique labels
unique_labels = sorted(list(set(y)))

plt.figure(figsize=(10, 7))
sns.heatmap(
    cm,
    annot=True,
    fmt=".2%",
    xticklabels=unique_labels,
    yticklabels=unique_labels,
    cmap="Blues",
)
plt.xlabel("Predicted")
plt.ylabel("Truth")
plt.show()

# Calculate precision and recall
precision = {}
recall = {}

for i in range(len(unique_labels)):
    cls = unique_labels[i]
    tp = cm[i, i]
    fp = np.sum(cm[:, i]) - tp
    fn = np.sum(cm[i, :]) - tp
    
    precision[cls] = tp / (tp + fp) if tp + fp > 0 else 0
    recall[cls] = tp / (tp + fn) if tp + fn > 0 else 0

print("Precision:\n", precision)
print("\nRecall:\n", recall)

tp_total = np.sum(np.diag(cm))
fp_total = np.sum(cm) - tp_total
fn_total = np.sum(cm) - tp_total

overall_precision = tp_total / (tp_total + fp_total) if tp_total + fp_total > 0 else 0
overall_recall = tp_total / (tp_total + fn_total) if tp_total + fn_total > 0 else 0

print("\nOverall precision:\n", overall_precision)
print("\nOverall recall:\n", overall_recall)